In [1]:
import numpy as np
import pandas as pd

In [2]:
HF_weather = pd.read_csv('../data/HF_weather/raw/hf001-08-hourly-m.csv', parse_dates=['datetime'])

/home/steffen/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,5,7,9,11,13,15,17,19,21,23,25,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
def get_gdd(data, tbase=5, datetime="datetime", airtemp="airt"):
    gdd_df = (
        data[[datetime, airtemp]]
        .groupby(pd.Grouper(key=datetime, freq="1D"))
        .mean()
        .reset_index()
    )
    gdd_df.rename(columns={airtemp: "mean_airt"}, inplace=True)
    gdd_df["GDD"] = gdd_df[["mean_airt"]].applymap(lambda x: 0 if x <= tbase else x - tbase)

    data_datetime = pd.DatetimeIndex(gdd_df[datetime])

    for year in data_datetime.year.unique():
        gdd_df.loc[(data_datetime.year == year), "cumGDD"] = gdd_df[
            data_datetime.year == year
        ]["GDD"].cumsum()

    return gdd_df

In [48]:
def get_frthw(data, threshold = 0, datetime="datetime", airtemp="airt"):
    
    # Initialize summary dataframe
    frthw_df = pd.DataFrame()
    
    data_datetime = pd.DatetimeIndex(data[datetime])
    data['frthw'] = np.nan
    
    for year in data_datetime.year.unique():
        
        year_data = data.loc[(data_datetime.year == year), ['datetime', 'airt','frthw']]
        frthw = 0.5 if year_data.iloc[0,1]>threshold else 0 # If first temp reading is above threshold, start frthw at 0.5
        
        for obs in range(year_data.shape[0]):
            if frthw%1 == 0:
                if year_data.iloc[obs,1] >= threshold:
                    frthw += 0.5
            else:
                if year_data.iloc[obs,1] < threshold:
                    frthw += 0.5
            year_data.iloc[obs,2] = frthw
        
        year_data = year_data.set_index("datetime").resample("1D").last()
        frthw_df = frthw_df.append(year_data)
        
    return frthw_df

In [52]:
gdd = get_gdd(HF_weather)
frzthw = get_frthw(HF_weather)
gdd_frthw = pd.concat([gdd,freezie.reset_index()['frthw']], axis =1)

In [60]:
gdd_frthw.to_pickle('../data/HF_weather/gdd_frthw')

,datetime,mean_airt,GDD,cumGDD,frthw
0,2005-01-01,3.784211,0.0,0.000000,1.0
1,2005-01-02,-2.167708,0.0,0.000000,1.5
2,2005-01-03,4.844792,0.0,0.000000,1.5
3,2005-01-04,2.445833,0.0,0.000000,2.0
4,2005-01-05,-2.596875,0.0,0.000000,2.0
...,...,...,...,...,...
5840,2020-12-28,1.571875,0.0,2328.147577,133.5
5841,2020-12-29,-3.317708,0.0,2328.147577,134.0
5842,2020-12-30,-4.055208,0.0,2328.147577,134.5
5843,2020-12-31,0.430208,0.0,2328.147577,136.0
